### Optimal Portfolio Allocation

**Question 6.** Re-implement backtesting using assumptions above (as necessary) and the EWMA variance forecast equation below. This is also known as RiskMetrics approach. 

The tutor will not reconfirm how to compute this model, but you can use the variance of your computed log-returns for the entire dataset to initialise the scheme. 

$$
\sigma_{t+1 | t} = \lambda \sigma_{t | t−1} + (1 − \lambda) r^2_t 
$$

with $\lambda = 0.72$ value set to minimise out of sample forecasting error, and $r_t$ refers to a return. 

Provide the same deliverables (a), (b), and (c) as in the previous Question.

### Answers and code

In [ ]:
# Import modules
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly.express as px

# Import plotly express
px.defaults.template = 'ggplot2'
px.defaults.width, px.defaults.height = 800, 600
import plotly.io as pio
pio.renderers.default='notebook'

In [ ]:
# Set probability value
c = 0.01
# Compute VaR factor
factor = norm.ppf(1 - c)
# import the data
df = pd.read_csv('Data_SP500.csv', index_col='Date')
# Compute the daily returns
df['Log Daily Return'] = np.log1p(df.pct_change(1))
# Compute the 10 day return
df['Log 10D Return'] = np.log1p(df['SP500'].pct_change(10))

# Compute rolling 21 day standard deviation for 1D and 10D
df['21D Rolling 1D SD'] = df['Log Daily Return'].rolling(21).std()
df['21D Rolling 10D SD'] = df['21D Rolling 1D SD'] * np.sqrt(10)

# Compute VaR for each row
df['VaR (Return)'] = -factor * df['21D Rolling 10D SD']
# Compute the breaches, 1 for yeas and 0 for no
df['Breach'] = df.apply(lambda row: 1 if (row['Log 10D Return'] < row['VaR (Return)']) else 0, axis=1)
# Drop rows that have NaN values
df.dropna(inplace=True)
# Compute consecutive breaches
df['Consecutive breach'] = df['Breach'] + df['Breach'].shift(1)
df['Consecutive breach'] = df['Consecutive breach'].apply(lambda row: 1 if (row > 1) else 0)

df.head()

In [ ]:
# Count the number and % of breaches
breaches = df['Breach'].sum()
percent_breaches = breaches / df['Breach'].count()
consecutive_breaches = df['Consecutive breach'].sum()
percent_consecutive_breaches = consecutive_breaches / df['Consecutive breach'].count()

print('Number of breaches: ', breaches)
print('% of breaches: ', f'{round(percent_breaches*100,3)}%')
print('Number of consecutive breaches: ', consecutive_breaches)
print('% of consecutive breaches: ', f'{round(percent_consecutive_breaches*100,3)}%')

In [ ]:
# Plot the results
fig = px.line(df['VaR (Return)'], y='VaR (Return)', x=df.index, title='Backtesting Value at Risk')
fig.update_traces(line_color='blue', opacity=0.5)
fig.add_bar(y=df['Log 10D Return'], x=df.index, name='10D Lognormal Return', marker_color='red', opacity=1)
fig.add_scatter(x=df[df['Breach'] > 0].index, y=df[df['Breach'] > 0]['Log 10D Return'], name='Breach', mode='markers', opacity=1)
fig.show()